# FizzBuzz

FizzBuzz是一个简单的小游戏。游戏规则如下：从1开始往上数数，当遇到3的倍数的时候，说fizz，当遇到5的倍数，说buzz，当遇到15的倍数，就说fizzbuzz，其他情况下则正常数数。

我们可以写一个简单的小程序来决定要返回正常数值还是fizz, buzz 或者 fizzbuzz。

In [1]:
# One-hot encode the desired outputs: [number, "fizz", "buzz", "fizzbuzz"]
def fizz_buzz_encode(i):
    if   i % 15 == 0: return 3
    elif i % 5  == 0: return 2
    elif i % 3  == 0: return 1
    else:             return 0
    
def fizz_buzz_decode(i, prediction):
    return [str(i), "fizz", "buzz", "fizzbuzz"][prediction]

print(fizz_buzz_decode(1, fizz_buzz_encode(1)))
print(fizz_buzz_decode(2, fizz_buzz_encode(2)))
print(fizz_buzz_decode(5, fizz_buzz_encode(5)))
print(fizz_buzz_decode(12, fizz_buzz_encode(12)))
print(fizz_buzz_decode(15, fizz_buzz_encode(15)))

1
2
buzz
fizz
fizzbuzz


我们首先定义模型的输入与输出(训练数据)

In [2]:
import numpy as np
import torch

NUM_DIGITS = 10

# Represent each input by an array of its binary digits.
def binary_encode(i, num_digits):
    return np.array([i >> d & 1 for d in range(num_digits)])

trX = torch.Tensor([binary_encode(i, NUM_DIGITS) for i in range(101, 2 ** NUM_DIGITS)])
trY = torch.LongTensor([fizz_buzz_encode(i) for i in range(101, 2 ** NUM_DIGITS)])

然后我们用PyTorch定义模型

In [3]:
# Define the model
NUM_HIDDEN = 100
model = torch.nn.Sequential(
    torch.nn.Linear(NUM_DIGITS, NUM_HIDDEN),
    torch.nn.ReLU(),
    torch.nn.Linear(NUM_HIDDEN, 4)
)

- 为了让我们的模型学会FizzBuzz这个游戏，我们需要定义一个损失函数，和一个优化算法。
- 这个优化算法会不断优化（降低）损失函数，使得模型的在该任务上取得尽可能低的损失值。
- 损失值低往往表示我们的模型表现好，损失值高表示我们的模型表现差。
- 由于FizzBuzz游戏本质上是一个分类问题，我们选用Cross Entropyy Loss函数。
- 优化函数我们选用Stochastic Gradient Descent。

In [4]:
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.05)

以下是模型的训练代码

In [5]:
# Start training it
BATCH_SIZE = 128
for epoch in range(10000):
    for start in range(0, len(trX), BATCH_SIZE):
        end = start + BATCH_SIZE
        batchX = trX[start:end]
        batchY = trY[start:end]

        y_pred = model(batchX)
        loss = loss_fn(y_pred, batchY)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Find loss on training data
    loss = loss_fn(model(trX), trY).item()
    print('Epoch:', epoch, 'Loss:', loss)

Epoch: 0 Loss: 1.197601556777954
Epoch: 1 Loss: 1.1617237329483032
Epoch: 2 Loss: 1.151560664176941
Epoch: 3 Loss: 1.147324800491333
Epoch: 4 Loss: 1.1451345682144165
Epoch: 5 Loss: 1.1438238620758057
Epoch: 6 Loss: 1.1429516077041626
Epoch: 7 Loss: 1.1423169374465942
Epoch: 8 Loss: 1.1418237686157227
Epoch: 9 Loss: 1.141418218612671
Epoch: 10 Loss: 1.141069769859314
Epoch: 11 Loss: 1.1407626867294312
Epoch: 12 Loss: 1.1404907703399658
Epoch: 13 Loss: 1.1402416229248047
Epoch: 14 Loss: 1.1400126218795776
Epoch: 15 Loss: 1.1398042440414429
Epoch: 16 Loss: 1.1396065950393677
Epoch: 17 Loss: 1.1394232511520386
Epoch: 18 Loss: 1.1392505168914795
Epoch: 19 Loss: 1.1390889883041382
Epoch: 20 Loss: 1.138929843902588
Epoch: 21 Loss: 1.138779640197754
Epoch: 22 Loss: 1.1386370658874512
Epoch: 23 Loss: 1.1384998559951782
Epoch: 24 Loss: 1.138366460800171
Epoch: 25 Loss: 1.1382390260696411
Epoch: 26 Loss: 1.1381171941757202
Epoch: 27 Loss: 1.1380021572113037
Epoch: 28 Loss: 1.1378891468048096
Epo

Epoch: 249 Loss: 1.0818804502487183
Epoch: 250 Loss: 1.0802106857299805
Epoch: 251 Loss: 1.0808403491973877
Epoch: 252 Loss: 1.079430341720581
Epoch: 253 Loss: 1.078934669494629
Epoch: 254 Loss: 1.0793161392211914
Epoch: 255 Loss: 1.0779776573181152
Epoch: 256 Loss: 1.0782697200775146
Epoch: 257 Loss: 1.0773881673812866
Epoch: 258 Loss: 1.0765308141708374
Epoch: 259 Loss: 1.0768948793411255
Epoch: 260 Loss: 1.0755106210708618
Epoch: 261 Loss: 1.075958490371704
Epoch: 262 Loss: 1.074486255645752
Epoch: 263 Loss: 1.0747525691986084
Epoch: 264 Loss: 1.073594331741333
Epoch: 265 Loss: 1.0736030340194702
Epoch: 266 Loss: 1.0722490549087524
Epoch: 267 Loss: 1.0716718435287476
Epoch: 268 Loss: 1.0726282596588135
Epoch: 269 Loss: 1.0708240270614624
Epoch: 270 Loss: 1.0714552402496338
Epoch: 271 Loss: 1.069410800933838
Epoch: 272 Loss: 1.0699942111968994
Epoch: 273 Loss: 1.0688867568969727
Epoch: 274 Loss: 1.0694667100906372
Epoch: 275 Loss: 1.068211555480957
Epoch: 276 Loss: 1.0678120851516724

Epoch: 484 Loss: 0.9542998671531677
Epoch: 485 Loss: 0.9540237188339233
Epoch: 486 Loss: 0.9525075554847717
Epoch: 487 Loss: 0.9525607824325562
Epoch: 488 Loss: 0.9515935182571411
Epoch: 489 Loss: 0.951810359954834
Epoch: 490 Loss: 0.9498243927955627
Epoch: 491 Loss: 0.9496254324913025
Epoch: 492 Loss: 0.9493669867515564
Epoch: 493 Loss: 0.9477329254150391
Epoch: 494 Loss: 0.9479847550392151
Epoch: 495 Loss: 0.9467636346817017
Epoch: 496 Loss: 0.9459443688392639
Epoch: 497 Loss: 0.9445229172706604
Epoch: 498 Loss: 0.9445067048072815
Epoch: 499 Loss: 0.9443970918655396
Epoch: 500 Loss: 0.9429887533187866
Epoch: 501 Loss: 0.9427738785743713
Epoch: 502 Loss: 0.9419804811477661
Epoch: 503 Loss: 0.9405749440193176
Epoch: 504 Loss: 0.9411666989326477
Epoch: 505 Loss: 0.9391089081764221
Epoch: 506 Loss: 0.939905047416687
Epoch: 507 Loss: 0.9379388689994812
Epoch: 508 Loss: 0.938086748123169
Epoch: 509 Loss: 0.936346173286438
Epoch: 510 Loss: 0.936650276184082
Epoch: 511 Loss: 0.93588590621948

Epoch: 746 Loss: 0.7115989923477173
Epoch: 747 Loss: 0.7108964920043945
Epoch: 748 Loss: 0.7083489894866943
Epoch: 749 Loss: 0.7092020511627197
Epoch: 750 Loss: 0.7080560326576233
Epoch: 751 Loss: 0.705642580986023
Epoch: 752 Loss: 0.7054673433303833
Epoch: 753 Loss: 0.704683244228363
Epoch: 754 Loss: 0.7029456496238708
Epoch: 755 Loss: 0.7033001184463501
Epoch: 756 Loss: 0.7010161280632019
Epoch: 757 Loss: 0.7014163732528687
Epoch: 758 Loss: 0.700290322303772
Epoch: 759 Loss: 0.6986166834831238
Epoch: 760 Loss: 0.6983052492141724
Epoch: 761 Loss: 0.6963403224945068
Epoch: 762 Loss: 0.6963163018226624
Epoch: 763 Loss: 0.6947327852249146
Epoch: 764 Loss: 0.6942377686500549
Epoch: 765 Loss: 0.692950963973999
Epoch: 766 Loss: 0.6917541027069092
Epoch: 767 Loss: 0.6912038922309875
Epoch: 768 Loss: 0.690735936164856
Epoch: 769 Loss: 0.6891598105430603
Epoch: 770 Loss: 0.688296914100647
Epoch: 771 Loss: 0.6876003742218018
Epoch: 772 Loss: 0.6859393119812012
Epoch: 773 Loss: 0.685438573360443

Epoch: 990 Loss: 0.5039297938346863
Epoch: 991 Loss: 0.5023817420005798
Epoch: 992 Loss: 0.5017673373222351
Epoch: 993 Loss: 0.5009730458259583
Epoch: 994 Loss: 0.5002804398536682
Epoch: 995 Loss: 0.49920788407325745
Epoch: 996 Loss: 0.49864864349365234
Epoch: 997 Loss: 0.49743378162384033
Epoch: 998 Loss: 0.4980713129043579
Epoch: 999 Loss: 0.49729418754577637
Epoch: 1000 Loss: 0.49521926045417786
Epoch: 1001 Loss: 0.49432945251464844
Epoch: 1002 Loss: 0.4949137568473816
Epoch: 1003 Loss: 0.4937874674797058
Epoch: 1004 Loss: 0.4926745891571045
Epoch: 1005 Loss: 0.49207910895347595
Epoch: 1006 Loss: 0.4916800856590271
Epoch: 1007 Loss: 0.49029335379600525
Epoch: 1008 Loss: 0.48959121108055115
Epoch: 1009 Loss: 0.4892214238643646
Epoch: 1010 Loss: 0.4893455505371094
Epoch: 1011 Loss: 0.48860064148902893
Epoch: 1012 Loss: 0.486531138420105
Epoch: 1013 Loss: 0.486398845911026
Epoch: 1014 Loss: 0.4851810336112976
Epoch: 1015 Loss: 0.4845695197582245
Epoch: 1016 Loss: 0.4839767515659332
Epo

...

Epoch: 9042 Loss: 0.008464348502457142
Epoch: 9043 Loss: 0.00846154522150755
Epoch: 9044 Loss: 0.008460883982479572
Epoch: 9045 Loss: 0.008458152413368225
Epoch: 9046 Loss: 0.008457120507955551
Epoch: 9047 Loss: 0.008455606177449226
Epoch: 9048 Loss: 0.008453966118395329
Epoch: 9049 Loss: 0.008451675064861774
Epoch: 9050 Loss: 0.008449958637356758
Epoch: 9051 Loss: 0.008449950255453587
Epoch: 9052 Loss: 0.0084479209035635
Epoch: 9053 Loss: 0.008446053601801395
Epoch: 9054 Loss: 0.008444482460618019
Epoch: 9055 Loss: 0.008443821221590042
Epoch: 9056 Loss: 0.008441876620054245
Epoch: 9057 Loss: 0.008439784869551659
Epoch: 9058 Loss: 0.008439659141004086
Epoch: 9059 Loss: 0.008436571806669235
Epoch: 9060 Loss: 0.008435703814029694
Epoch: 9061 Loss: 0.008433920331299305
Epoch: 9062 Loss: 0.008431477472186089
Epoch: 9063 Loss: 0.008430748246610165
Epoch: 9064 Loss: 0.008429191075265408
Epoch: 9065 Loss: 0.008427953347563744
Epoch: 9066 Loss: 0.008425663225352764
Epoch: 9067 Loss: 0.00842400

Epoch: 9271 Loss: 0.008119226433336735
Epoch: 9272 Loss: 0.008117263205349445
Epoch: 9273 Loss: 0.008116467855870724
Epoch: 9274 Loss: 0.008114781230688095
Epoch: 9275 Loss: 0.008114371448755264
Epoch: 9276 Loss: 0.008112412877380848
Epoch: 9277 Loss: 0.008110535331070423
Epoch: 9278 Loss: 0.008109494112432003
Epoch: 9279 Loss: 0.008107401430606842
Epoch: 9280 Loss: 0.008106714114546776
Epoch: 9281 Loss: 0.008104912005364895
Epoch: 9282 Loss: 0.008103489875793457
Epoch: 9283 Loss: 0.0081021161749959
Epoch: 9284 Loss: 0.008100165985524654
Epoch: 9285 Loss: 0.008099047467112541
Epoch: 9286 Loss: 0.008096650242805481
Epoch: 9287 Loss: 0.008096562698483467
Epoch: 9288 Loss: 0.0080941841006279
Epoch: 9289 Loss: 0.008092956617474556
Epoch: 9290 Loss: 0.008092491887509823
Epoch: 9291 Loss: 0.008089940994977951
Epoch: 9292 Loss: 0.008089604787528515
Epoch: 9293 Loss: 0.008087721653282642
Epoch: 9294 Loss: 0.008086460642516613
Epoch: 9295 Loss: 0.00808387529104948
Epoch: 9296 Loss: 0.0080834040

Epoch: 9491 Loss: 0.007810922339558601
Epoch: 9492 Loss: 0.007809273898601532
Epoch: 9493 Loss: 0.007808591704815626
Epoch: 9494 Loss: 0.0078058368526399136
Epoch: 9495 Loss: 0.007805380504578352
Epoch: 9496 Loss: 0.007804572116583586
Epoch: 9497 Loss: 0.007802835665643215
Epoch: 9498 Loss: 0.007801094092428684
Epoch: 9499 Loss: 0.007799688261002302
Epoch: 9500 Loss: 0.007798745762556791
Epoch: 9501 Loss: 0.007797393016517162
Epoch: 9502 Loss: 0.0077959815971553326
Epoch: 9503 Loss: 0.007794214878231287
Epoch: 9504 Loss: 0.00779333058744669
Epoch: 9505 Loss: 0.007791532669216394
Epoch: 9506 Loss: 0.007790728006511927
Epoch: 9507 Loss: 0.007788925431668758
Epoch: 9508 Loss: 0.007787459529936314
Epoch: 9509 Loss: 0.007786520291119814
Epoch: 9510 Loss: 0.007785836234688759
Epoch: 9511 Loss: 0.007783746346831322
Epoch: 9512 Loss: 0.007782517001032829
Epoch: 9513 Loss: 0.007781436201184988
Epoch: 9514 Loss: 0.007779353763908148
Epoch: 9515 Loss: 0.007778739091008902
Epoch: 9516 Loss: 0.0077

Epoch: 9708 Loss: 0.007526460103690624
Epoch: 9709 Loss: 0.007525453809648752
Epoch: 9710 Loss: 0.007524343207478523
Epoch: 9711 Loss: 0.007523390464484692
Epoch: 9712 Loss: 0.007522110361605883
Epoch: 9713 Loss: 0.007520444691181183
Epoch: 9714 Loss: 0.007518904283642769
Epoch: 9715 Loss: 0.007518306374549866
Epoch: 9716 Loss: 0.0075163086876273155
Epoch: 9717 Loss: 0.007514801807701588
Epoch: 9718 Loss: 0.007514207158237696
Epoch: 9719 Loss: 0.007512430660426617
Epoch: 9720 Loss: 0.007510633673518896
Epoch: 9721 Loss: 0.00751106021925807
Epoch: 9722 Loss: 0.007509159855544567
Epoch: 9723 Loss: 0.007507489062845707
Epoch: 9724 Loss: 0.0075069814920425415
Epoch: 9725 Loss: 0.007504604756832123
Epoch: 9726 Loss: 0.00750383734703064
Epoch: 9727 Loss: 0.007502538152039051
Epoch: 9728 Loss: 0.007501280400902033
Epoch: 9729 Loss: 0.007499950937926769
Epoch: 9730 Loss: 0.007498853839933872
Epoch: 9731 Loss: 0.007498233113437891
Epoch: 9732 Loss: 0.0074970917776227
Epoch: 9733 Loss: 0.0074954

Epoch: 9947 Loss: 0.007235150784254074
Epoch: 9948 Loss: 0.007233970798552036
Epoch: 9949 Loss: 0.007232500705868006
Epoch: 9950 Loss: 0.007231953553855419
Epoch: 9951 Loss: 0.007230811752378941
Epoch: 9952 Loss: 0.007228840608149767
Epoch: 9953 Loss: 0.007227961905300617
Epoch: 9954 Loss: 0.007227203343063593
Epoch: 9955 Loss: 0.007225691340863705
Epoch: 9956 Loss: 0.007224895525723696
Epoch: 9957 Loss: 0.007223495747894049
Epoch: 9958 Loss: 0.00722165172919631
Epoch: 9959 Loss: 0.0072216615080833435
Epoch: 9960 Loss: 0.007219857536256313
Epoch: 9961 Loss: 0.00721786031499505
Epoch: 9962 Loss: 0.0072179832495749
Epoch: 9963 Loss: 0.007216674741357565
Epoch: 9964 Loss: 0.007214795332401991
Epoch: 9965 Loss: 0.00721382349729538
Epoch: 9966 Loss: 0.007212954107671976
Epoch: 9967 Loss: 0.0072111766785383224
Epoch: 9968 Loss: 0.007210583426058292
Epoch: 9969 Loss: 0.007209564559161663
Epoch: 9970 Loss: 0.007207964081317186
Epoch: 9971 Loss: 0.007207140326499939
Epoch: 9972 Loss: 0.00720552

最后我们用训练好的模型尝试在1到100这些数字上玩FizzBuzz游戏

In [7]:
# Output now
testX = torch.Tensor([binary_encode(i, NUM_DIGITS) for i in range(1, 101)])
with torch.no_grad():
    testY = model(testX)
predictions = zip(range(1, 101), list(testY.max(1)[1].data.tolist()))

print([fizz_buzz_decode(i, x) for (i, x) in predictions])

['1', '2', 'fizz', '4', 'buzz', 'fizz', '7', '8', 'fizz', 'buzz', '11', 'fizz', '13', '14', 'fizzbuzz', '16', '17', 'fizz', '19', 'buzz', 'fizz', '22', '23', 'fizz', '25', '26', 'fizz', '28', '29', 'fizzbuzz', '31', '32', 'fizz', '34', 'buzz', 'fizz', '37', '38', 'fizz', 'buzz', '41', 'fizz', '43', '44', 'fizzbuzz', '46', '47', 'fizz', '49', 'buzz', 'fizz', '52', '53', 'fizz', 'buzz', '56', 'fizz', '58', '59', 'fizzbuzz', '61', '62', 'fizz', '64', 'buzz', 'fizz', '67', 'buzz', 'fizz', 'buzz', '71', 'fizz', '73', '74', 'fizzbuzz', '76', '77', 'fizz', '79', 'buzz', 'fizz', '82', '83', 'fizz', 'buzz', '86', 'fizz', '88', '89', 'fizzbuzz', '91', '92', '93', '94', 'buzz', 'fizz', '97', '98', 'fizz', 'buzz']


In [8]:
print(np.sum(testY.max(1)[1].numpy() == np.array([fizz_buzz_encode(i) for i in range(1,101)])))
testY.max(1)[1].numpy() == np.array([fizz_buzz_encode(i) for i in range(1,101)])

97


array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True, False,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True, False,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True, False,  True,  True,  True,  True,  True,  True,
        True])